In [1]:
# Convolutional Neural Network
# keras functional API
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input,Layer
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras import backend as K

import tensorflow as tf
tf.set_random_seed(1)

/Users/zukako/.pyenv/versions/anaconda3-4.0.0/envs/keras/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
ALPHA = 0.2  # Triplet Loss Parameter

imhight = 128
imwidth = 128

# ネットワーク構造の定義
低層を共有

In [3]:
# shared Layer
shared_Conv1= Conv2D(64, (4,4), padding='same', activation='relu')
shared_MP1= MaxPooling2D(pool_size=(2, 2))

In [4]:
# anchor 
a_in = Input(shape = (imhight, imwidth,3), name='anchor_input')
conv1 = shared_Conv1(a_in)
pool1 = shared_MP1(conv1)
conv2 = Conv2D(32, (4,4) , padding='same', activation='relu')(pool1)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
dense = Dense(20, activation='relu')(pool2)
a_emb = Flatten(name='anchor_output')(dense)

In [5]:
# positive sample
p_in = Input(shape = (imhight, imwidth,3), name='positive_input')
conv1 =  shared_Conv1(p_in)
pool1 = shared_MP1(conv1)
conv2 = Conv2D(32, (4,4) , padding='same', activation='relu')(pool1)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
dense = Dense(20, activation='relu')(pool2)
p_emb = Flatten(name='positive_output')(dense)

In [6]:
# negative sample
n_in = Input(shape = (imhight, imwidth,3))
conv1 =  shared_Conv1(n_in)
pool1 = shared_MP1(conv1)
conv2 = Conv2D(32, (4,4) , padding='same', activation='relu')(pool1)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
dense = Dense(20, activation='relu')(pool2)
n_emb = Flatten(name='negative_output')(dense)

## triplet loss計算用のLayerインスタンスをLayerクラスを継承して作成
tensorflow -> numpy ×<br>
numpy -> tensorflow ○<br>
triplet_lossの計算には **tensorflowの書式** を用いる。

In [7]:
class TripletLossLayer(Layer):
    def __init__(self, alpha, **kwargs):
        self.alpha = alpha
        super(TripletLossLayer, self).__init__(**kwargs)
    
    def triplet_loss(self, inputs):
        a, p, n = inputs
        p_dist = K.sum(K.square(a-p), axis=-1)
        n_dist = K.sum(K.square(a-n), axis=-1)
        return K.sum(K.maximum(p_dist - n_dist + self.alpha, 0), axis=0)
    
    def call(self, inputs):
        loss = self.triplet_loss(inputs)
        self.add_loss(loss)
        return loss

・ [a_emb,p_emb,n_emb]のtripletを入力とするレイヤーを定義<br>
・ LayerインスタンスとしてModelの中に組み込む<br>
・ Inputインスタンスを指定すると、emb(埋込)までを認識してくれる。

In [8]:
# Layer that computes the triplet loss from anchor, positive and negative embedding vectors
triplet_loss_layer = TripletLossLayer(alpha=0.2, name='triplet_loss_layer')([a_emb, p_emb, n_emb])

# Model that can be trained with anchor, positive negative images
tripletNet = Model([a_in, p_in, n_in], triplet_loss_layer)

ネットワーク構造のプロット

In [9]:
plot_model(tripletNet, to_file='tripletNet.png')

## 動きの確認

In [10]:
# sample data
# data_gen.pyからtriplet_generatorを読み込んでsample生成
from data_gen import triplet_generator
generator = triplet_generator()

tripletNet.compile(loss=None, optimizer='adam')
tripletNet.fit_generator(generator=generator, epochs=10, steps_per_epoch=100)

/Users/zukako/.pyenv/versions/anaconda3-4.0.0/envs/keras/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Output "triplet_loss_layer" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "triplet_loss_layer" during training.
  """


Epoch 1/10
100/100 [==============================] - 59s 589ms/step - loss: 0.0000e+00
Epoch 2/10
100/100 [==============================] - 59s 590ms/step - loss: 0.0000e+00
Epoch 3/10
100/100 [==============================] - 59s 588ms/step - loss: 0.0000e+00
Epoch 4/10
100/100 [==============================] - 4589s 46s/step - loss: 0.0000e+00
Epoch 5/10
100/100 [==============================] - 61s 612ms/step - loss: 0.0000e+00
Epoch 6/10
100/100 [==============================] - 61s 614ms/step - loss: 0.0000e+00
Epoch 7/10
100/100 [==============================] - 61s 613ms/step - loss: 0.0000e+00
Epoch 8/10
100/100 [==============================] - 61s 606ms/step - loss: 0.0000e+00
Epoch 9/10
100/100 [==============================] - 345s 3s/step - loss: 0.0000e+00
Epoch 10/10
100/100 [==============================] - 65s 652ms/step - loss: 0.0000e+00


In [18]:
import numpy as np
a_batch = np.random.rand(4, 128, 128, 3)
p_batch = np.random.rand(4, 128, 128, 3)
n_batch = np.random.rand(4, 128, 128, 3)

x = [a_batch,p_batch,n_batch]

In [19]:
tripletNet.fit(x, epochs=10, steps_per_epoch=100)

Epoch 1/10
100/100 [==============================] - 58s 581ms/step - loss: 0.0000e+00
Epoch 2/10
100/100 [==============================] - 61s 612ms/step - loss: 0.0000e+00
Epoch 3/10
100/100 [==============================] - 59s 592ms/step - loss: 0.0000e+00
Epoch 4/10
100/100 [==============================] - 65s 647ms/step - loss: 0.0000e+00
Epoch 5/10
100/100 [==============================] - 74s 735ms/step - loss: 0.0000e+00
Epoch 6/10
100/100 [==============================] - 80s 802ms/step - loss: 0.0000e+00
Epoch 7/10
100/100 [==============================] - 85s 849ms/step - loss: 0.0000e+00
Epoch 8/10
100/100 [==============================] - 89s 893ms/step - loss: 0.0000e+00
Epoch 9/10
100/100 [==============================] - 81s 814ms/step - loss: 0.0000e+00
Epoch 10/10
100/100 [==============================] - 92s 923ms/step - loss: 0.0000e+00
